In [289]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import random

import os
from obspy.core import read

from sklearn.model_selection import train_test_split

### EVENTS

In [290]:
main_path = os.path.abspath("")

In [291]:
#DATAFRAME WITH ALL EVENTS
all_events_file_path = os.path.join(main_path, 'earthquakes_filtered.txt') #all events
all_events = pd.read_csv(all_events_file_path, sep=',')

try:
    all_events.drop(columns=['Unnamed: 0'], inplace=True) #automatically created column (idk why)
except:
    pass

all_events.head()

,event_id,event_ID,year,month,day,hour,minute,second,lat,lng,depth,mag_ML,std_dev_ML,mag_MA,std_dev_MA,category
0,0,0,2007,1,1,2,41,13.28,-21.65559,-68.41471,121.33,2.345,0.020,2.394,0.029,0
1,1,1,2007,1,1,2,47,7.83,-20.54848,-69.05857,102.79,1.114,0.033,1.305,0.031,0
2,2,2,2007,1,1,3,50,29.15,-21.86299,-68.53639,110.95,2.779,0.031,2.917,0.031,0
3,3,3,2007,1,1,4,19,27.82,-20.29515,-69.13106,95.79,1.401,0.017,1.571,0.023,0
4,4,4,2007,1,1,5,40,2.58,-21.23847,-70.05151,34.64,1.995,0.022,2.222,0.018,0


In [292]:
#DATAFRAME FROM THE EXISTING FILES IN geofon_waveforms FOLDER
dataset_size = 10000

file_list = os.listdir(os.path.join(main_path, "geofon_waveforms"))
file_list = [int(file[:-6]) for file in file_list] #remove the '.mseed' ending and convert to int to get event_id
file_list = random.sample(file_list, dataset_size) #select a random sample of N events from all the files

train_events, test_events = train_test_split(file_list, test_size=0.2, random_state=42) #split the dataset into the training and testing part

train_events = pd.DataFrame(data = train_events, columns = ['event_id'])
train_events = pd.merge(left = train_events, right = all_events, on='event_id', how= 'inner')

test_events = pd.DataFrame(data = test_events, columns = ['event_id'])
test_events = pd.merge(left = test_events, right = all_events, on='event_id', how= 'inner')

In [293]:
#TRAIN EVENTS
train_events.head()

,event_id,event_ID,year,month,day,hour,minute,second,lat,lng,depth,mag_ML,std_dev_ML,mag_MA,std_dev_MA,category
0,51663,53059,2010,12,7,9,33,16.59,-20.69449,-68.81573,101.17,2.135,0.017,2.276,0.018,0
1,9599,9841,2007,10,20,5,48,1.91,-21.25235,-68.76697,107.88,1.381,0.022,1.518,0.020,0
2,35530,36421,2009,8,28,11,8,24.64,-20.61351,-69.84340,60.38,1.732,0.030,2.003,0.038,0
3,32271,33049,2009,5,28,4,29,49.30,-21.10066,-68.08070,142.93,2.097,0.015,2.092,0.013,0
4,11150,11427,2007,11,19,3,6,7.78,-22.39223,-70.06235,46.26,0.692,0.019,0.952,0.016,0


In [294]:
#TEST EVENTS
test_events.head()

,event_id,event_ID,year,month,day,hour,minute,second,lat,lng,depth,mag_ML,std_dev_ML,mag_MA,std_dev_MA,category
0,34338,35190,2009,7,21,21,36,52.00,-21.09701,-68.79828,113.36,1.941,0.024,2.027,0.020,0
1,36338,37250,2009,9,22,3,40,36.87,-21.52173,-68.37471,130.73,2.811,0.029,2.821,0.032,0
2,41000,42053,2010,2,6,16,27,44.40,-20.99417,-69.59231,11.24,1.354,0.022,1.591,0.030,1
3,40124,41152,2010,1,11,11,43,6.75,-20.74872,-68.85488,94.79,1.183,0.021,1.367,0.017,0
4,47820,49103,2010,8,20,6,39,46.60,-21.09078,-68.45683,111.81,1.721,0.021,1.817,0.020,0


### CUSTOM DATASET

In [295]:
class event_dataset(Dataset):
    def __init__(self, dataset_type: str, transform = None) -> None:

        if dataset_type not in ['train', 'test']:
            raise KeyError("dataset_type has to be one of the follwoing: 'train', 'test' ")

        if dataset_type == "train":
            self.dataframe = train_events
        elif dataset_type == "test":
            self.dataframe = test_events

        self.data_direcotry = "geofon_waveforms"
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        event_id, label = int(row['event_id']), int(row['category'])

        #WAVEFORM FETCH
        file_name = f"{event_id}.mseed"
        waveform = read(os.path.join(main_path, self.data_direcotry, file_name))

        #WAVEFORM PREP
        waveform = [trace.data for trace in waveform]
        waveform = np.stack(waveform, axis = 0, dtype=np.float32) #stack arrays such that we have 4801 arrays of len(3) - as if it is rgb
        
        #WRITE AS TENSORS
        label = torch.tensor(data= label, dtype= torch.int64)
        waveform = torch.from_numpy(waveform)

        #CREATE SAMPLE
        sample = {'label': label,
                  'data': waveform}
        
        if self.transform:
            sample['data'] = self.transform(sample['data'])

        return sample


In [296]:
# row = train_events.iloc[650]
# event_id, label = int(row['event_id']), int(row['category'])
# file_name = f"{event_id}.mseed"
# waveform = read(os.path.join(main_path, "geofon_waveforms", file_name))
# waveform = [trace.data for trace in waveform]
# waveform = np.stack(waveform, axis = 0, dtype=np.float32)
# waveform = torch.from_numpy(waveform)
# label = torch.tensor(data= label, dtype= torch.int64)
# waveform.shape, label.shape
# waveform_n = nn.functional.normalize(input=waveform)
# waveform_n

In [297]:
# max_pool = nn.MaxPool1d(kernel_size=2, stride = 2)
# relu = nn.ReLU()
# conv1 = nn.Conv1d(in_channels = 3, out_channels = 64, kernel_size = 50)
# wt1 = relu(conv1(waveform_n))
# print(wt1, wt1.shape)
# wt1 = max_pool(wt1)
# print(wt1, wt1.shape)

# conv2 = nn.Conv1d(in_channels = 64, out_channels = 1, kernel_size = 5)
# wt2 = relu(conv2(wt1))
# print(wt2, wt2.shape)
# wt2 = max_pool(wt2)
# print(wt2, wt2.shape)

# fc1 = nn.Linear(in_features= 1186 , out_features=64)
# wt3 = relu(fc1(wt2))
# print(wt3, wt3.shape)

# sig = nn.Sigmoid()
# fc2 = nn.Linear(in_features= 64, out_features=1)
# wt4 = sig(fc2(wt3))
# print(wt4, wt4.shape)

### CNN MODEL

In [298]:
class seismic_CNN(nn.Module):
    def __init__(self) -> None:
        super(seismic_CNN, self).__init__()
        self.max_pool = nn.MaxPool1d(kernel_size=2, stride = 2)

        self.conv1 = nn.Conv1d(in_channels = 3, out_channels = 64, kernel_size = 50)
        self.conv2 = nn.Conv1d(in_channels = 64, out_channels = 1, kernel_size = 5)
      
        self.fc1 = nn.Linear(in_features= 1186 , out_features=64)
        self.fc2 = nn.Linear(in_features= 64, out_features= 1)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)

        x = F.relu(self.conv2(x))
        x = self.max_pool(x)

        x = F.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))

        return x


### BASIC CNN  AND OTHER PARAMETERS

In [299]:
batch_size = 100
num_epochs = 10
learning_rate = 0.01
momentum = 0.9

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = seismic_CNN().to(device=device)
loss_funciton = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr = learning_rate,
                            momentum=momentum)

### CNN TRAINING

In [301]:
train_loader = DataLoader(dataset = event_dataset(dataset_type='train', transform=nn.functional.normalize),
                          batch_size = batch_size,
                          shuffle=True,
                          num_workers=0)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for idx, sample in enumerate(train_loader):
        #load data
        labels = sample['label'].to(device)
        data = sample['data'].to(device)

        #forward pass
        labels_logit = model(data).squeeze()
        labels_pred = torch.round(torch.sigmoid(labels_logit))

        #calculate loss / accuracy
        loss = loss_funciton(labels_pred, labels.float())
       
        #optimizer zero grad
        optimizer.zero_grad()

        #loss backwards
        loss.backward()
        
        #optimizer step
        optimizer.step()

        print (f'Epoch [{epoch+1}/{num_epochs}], Step [{idx+1}/{n_total_steps}], Loss: {loss:.4f}')

print('Finished Training')
PATH = './seismic_cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/10], Step [1/80], Loss: 1.2033
Epoch [1/10], Step [2/80], Loss: 1.1933
Epoch [1/10], Step [3/80], Loss: 1.2133
Epoch [1/10], Step [4/80], Loss: 1.2733
Epoch [1/10], Step [5/80], Loss: 1.2233
Epoch [1/10], Step [6/80], Loss: 1.2233
Epoch [1/10], Step [7/80], Loss: 1.2433
Epoch [1/10], Step [8/80], Loss: 1.1833
Epoch [1/10], Step [9/80], Loss: 1.2333
Epoch [1/10], Step [10/80], Loss: 1.2633
Epoch [1/10], Step [11/80], Loss: 1.2033
Epoch [1/10], Step [12/80], Loss: 1.1833
Epoch [1/10], Step [13/80], Loss: 1.2433
Epoch [1/10], Step [14/80], Loss: 1.2133
Epoch [1/10], Step [15/80], Loss: 1.2233
Epoch [1/10], Step [16/80], Loss: 1.2233
Epoch [1/10], Step [17/80], Loss: 1.2333
Epoch [1/10], Step [18/80], Loss: 1.2633
Epoch [1/10], Step [19/80], Loss: 1.2233
Epoch [1/10], Step [20/80], Loss: 1.2533
Epoch [1/10], Step [21/80], Loss: 1.2933
Epoch [1/10], Step [22/80], Loss: 1.2033
Epoch [1/10], Step [23/80], Loss: 1.2633
Epoch [1/10], Step [24/80], Loss: 1.2533
Epoch [1/10], Step [25/80

### CNN TESTING

In [302]:
model = seismic_CNN()
model.load_state_dict(torch.load(PATH))

test_loader = torch.utils.data.DataLoader(dataset = event_dataset(dataset_type='test'),
                                          batch_size=batch_size,
                                            shuffle=False)
n_train_samples = len(test_loader)
overall_correct_guesses = 0

with torch.inference_mode():
    for sample in test_loader:
        data = sample['data'].to(device)
        labels = sample['label'].to(device)
        print(f"actual labels: {labels}")
        
        test_logits = model(data).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        print(f"predicted labels: {test_pred}")

        correct_guesses = torch.eq(labels, labels_pred).sum().item()
        overall_correct_guesses += correct_guesses
        print(f"number of correct guesses: {correct_guesses}")

print(f"Model accuracy: {round(overall_correct_guesses/n_train_samples, 2)}%")
        

actual labels: tensor([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0])
predicted labels: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1.])
number of correct guesses: 8
actual labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 